In [ ]:
!pip install estnltk==1.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.2/222.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

from estnltk import Text

from nltk.util import ngrams
from nltk.probability import *

In [ ]:
# connect to drive
from google.colab import drive
from google.colab import files

drive.mount("/content/drive")
data = pd.read_csv("/content/drive/MyDrive/EstonianStanceDetection/notebooks/data.csv")

data["morph_analysis"] = data["sentence"].apply(lambda s: Text(s).tag_layer())

for feature in ["named_entities", "noun_phrases", "adjectives", "quoted_words", "diminutives", "superlatives", "conditionals", "translatives", "indirects", "stopwords", "unique_stopwords"]:
  data[feature] = data[feature].apply(eval)

Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# local
data = pd.read_csv("data.csv")

# Bigram Analysis

In [ ]:
url_stopwords = "https://raw.githubusercontent.com/kristel-/estonian-stopwords/master/estonian-stopwords.txt"
stopwords = pd.read_csv(url_stopwords, header=None, names=['stopword'])
stopword_list = list(stopwords.stopword)

bw_words = ["kõik", "kõige", "kunagi", "eales", "iial", "alati", "igavesti", "tervenisti", "täiesti", "üleni", "täitsa", "täielikult", "üdini", "läbinisti", "läbini", "absoluutne", "absoluutselt", "totaalne", "totaalselt", "ainult", "ainus", "kogu"]

for word in bw_words:
  if word in stopword_list:
    stopword_list.remove(word)

In [ ]:
def get_ngrams(morph_layer, n=2):
  lemmas = list(morph_layer.lemma)
  lemma_list = [lemma[0].lower() for lemma in lemmas if lemma[0] not in stopword_list and len(lemma[0]) >= 2]
  res = list(ngrams(lemma_list, n))
  return res

In [ ]:
against_bigrams = []
neutral_bigrams = []
support_bigrams = []

for i, row in data.iterrows():
  bigrams = get_ngrams(row["morph_analysis"], 2)
  if row["stance"] == "against":
    against_bigrams += bigrams
  elif row["stance"] == "neutral":
    neutral_bigrams += bigrams
  elif row["stance"] == "supportive":
    support_bigrams += bigrams

In [ ]:
bi_freq_against = FreqDist(against_bigrams).most_common()
bi_freq_neutral = FreqDist(neutral_bigrams).most_common()
bi_freq_support = FreqDist(support_bigrams).most_common()

In [ ]:
bi_against = []
for bigram, count in bi_freq_against:
  if count >= 5:
    print(f"{count: <3} {bigram}")
    bi_against += [" ".join(bigram)]

38  ('euroopa', 'liit')
18  ('mart', 'helme')
18  ('eesti', 'keel')
14  ('konservatiivne', 'rahvaerakond')
13  ('illegaalne', 'immigrant')
12  ('kogu', 'euroopa')
11  ('martin', 'helme')
11  ('tooma', 'kaasa')
10  ('eesti', 'konservatiivne')
10  ('neeger', 'araablane')
10  ('helme', 'sõna')
10  ('2015.', 'aasta')
9   ('kogu', 'maailm')
9   ('eesti', 'riik')
8   ('pagulane', 'vastuvõtmine')
8   ('eesti', 'rahvas')
8   ('euroopa', 'komisjon')
8   ('odav', 'tööjõud')
8   ('2016.', 'aasta')
8   ('donald', 'trump')
7   ('uus', 'uudis')
6   ('üro', 'rändelepe')
6   ('pagulane', 'võtma')
6   ('eesti', 'valitsus')
6   ('ütlema', 'helme')
6   ('el', 'liikmesriik')
6   ('eesti', 'euroopa')
6   ('itaalia', 'sadam')
6   ('neeger', 'moslem')
6   ('massiline', 'sisseränne')
6   ('illegaalne', 'immigratsioon')
6   ('värk', 'ütlema')
6   ('välismaalane', 'seadus')
6   ('kaasa', 'tooma')
6   ('usa', 'president')
6   ('helme', '.„')
5   ('globalism', 'multikultuursus')
5   ('kolmas', 'maailm')
5   ('kee

In [ ]:
bi_support = []
for bigram, count in bi_freq_support:
  if count >= 5:
    print(f"{count: <3} {bigram}")
    bi_support += [" ".join(bigram)]

29  ('euroopa', 'liit')
11  ('eesti', 'keel')
8   ('euroopa', 'komisjon')
7   ('miljon', 'euro')
6   ('välismaalane', 'seadus')
6   ('süüria', 'põgenik')
5   ('eesti', 'pagulasabi')
5   ('aafrika', 'päritolu')
5   ('sisseränne', 'piirarv')
5   ('globaalne', 'ränderaamistik')
5   ('rahvusvaheline', 'kaitse')


In [ ]:
bi_neutral = []
for bigram, count in bi_freq_neutral:
  if count >= 5:
    print(f"{count: <3} {bigram}")
    bi_neutral += [" ".join(bigram)]

104 ('euroopa', 'liit')
25  ('politsei', 'piirivalveamet')
19  ('2015.', 'aasta')
15  ('euroopa', 'komisjon')
14  ('euroopa', 'riik')
13  ('illegaalne', 'immigrant')
13  ('välismaalane', 'seadus')
13  ('illegaalne', 'immigratsioon')
12  ('tähtajaline', 'elamisluba')
11  ('suur', 'osa')
11  ('2017.', 'aasta')
11  ('ebaseaduslik', 'ränne')
11  ('piirivalveamet', 'ppa')
11  ('2016.', 'aasta')
11  ('sisseränne', 'piirarv')
10  ('viimane', 'aasta')
9   ('2014.', 'aasta')
9   ('kümme', 'tuhat')
8   ('eesti', 'riik')
8   ('eesti', 'ühiskond')
8   ('aasta', 'lõpp')
8   ('lühiajaline', 'töötamine')
8   ('islamiriik', 'võitleja')
8   ('mart', 'helme')
8   ('eesti', 'kodanik')
8   ('2018.', 'aasta')
7   ('pagulane', 'vastuvõtmine')
7   ('tuhat', 'migrant')
7   ('liit', 'riik')
7   ('aitama', 'kaasa')
7   ('kreeka', 'saar')
7   ('varjupaigataotleja', 'majutuskeskus')
7   ('riik', 'kodanik')
7   ('rahvusvaheline', 'kaitse')
7   ('kolmas', 'riik')
7   ('välismaalane', 'eesti')
7   ('itaalia', 'kreek

In [ ]:
fil_bg_against = [bigram for bigram in bi_against if bigram not in bi_neutral and bigram not in bi_support]
fil_bg_support = [bigram for bigram in bi_support if bigram not in bi_neutral and bigram not in bi_against]

In [ ]:
def contains_bigram(morph_layer, bigrams=[]):
  lemmas = list(morph_layer.lemma)
  lemma_list = [lemma[0].lower() for lemma in lemmas if lemma[0] not in stopword_list and len(lemma[0]) >= 2]
  lemma_string = " ".join(lemma_list)

  for pair in bigrams:
    if pair in lemma_string:
      return 1

  return 0

In [ ]:
data["has_against_bigram"] = data["morph_analysis"].apply(contains_bigram, bigrams=fil_bg_against)
data["has_support_bigram"] = data["morph_analysis"].apply(contains_bigram, bigrams=fil_bg_support)

In [ ]:
data[data["has_against_bigram"] == 1]["stance"].value_counts()

stance
against       193
neutral        59
supportive     25
Name: count, dtype: int64

In [ ]:
data[data["has_support_bigram"] == 1]["stance"].value_counts()

stance
supportive    19
neutral       12
Name: count, dtype: int64

#Adjective-based Framing

In [ ]:
adj_noun_list = []

for i, row in data.iterrows():

  if row["stanceConsolidated"] != 3:
    continue

  prev_pos, prev_word = "-", "-"
  for pos, word in zip(row["morph_analysis"].partofspeech, row["morph_analysis"].words.text):
    pos = pos[0]
    word = word.lower()
    if prev_pos in "ACU" and pos in "S":
      adj_noun_list += [(prev_word, word)]
    prev_pos = pos
    prev_word = word

neutral_pairs = FreqDist(adj_noun_list)

In [ ]:
for pair, count in neutral_pairs.most_common():
  print(f"{count: <3} {pair}")

10  ('eelmisel', 'nädalal')
8   ('eelmisel', 'aastal')
7   ('viimastel', 'aastatel')
7   ('tähtajalise', 'elamisloa')
6   ('ebaseadusliku', 'rände')
6   ('suur', 'osa')
6   ('lühiajalise', 'töötamise')
6   ('rahvusvahelise', 'kaitse')
6   ('illegaalse', 'immigratsiooni')
5   ('viimasel', 'ajal')
5   ('viimastel', 'kuudel')
4   ('eelmise', 'aasta')
4   ('suurem', 'osa')
4   ('kogu', 'aeg')
3   ('keskmist', 'brutokuupalka')
3   ('ebaseadusliku', 'sisserände')
3   ('eitava', 'vastuse')
3   ('illegaalsele', 'immigratsioonile')
3   ('terve', 'mõistuse')
2   ('suure', 'osa')
2   ('ebaseaduslike', 'migrantide')
2   ('kiiremas', 'korras')
2   ('noorte', 'meeste')
2   ('tähtajalist', 'elamisluba')
2   ('saabunud', 'riiki')
2   ('suurel', 'hulgal')
2   ('loomuliku', 'iibe')
2   ('võimaliku', 'rändekriisi')
2   ('viibivad', 'välismaalased')
2   ('rahvusvahelist', 'õigust')
2   ('praegusel', 'hetkel')
2   ('illegaalsed', 'immigrandid')
2   ('puudutavaid', 'ümberkorraldusi')
2   ('eelmise', 'nädala

In [ ]:
adj_noun_list = []

for i, row in data.iterrows():

  if row["stanceConsolidated"] >= 3:
    continue

  prev_pos, prev_word = "-", "-"
  for pos, word in zip(row["morph_analysis"].partofspeech, row["morph_analysis"].words.text):
    pos = pos[0]
    word = word.lower()
    if prev_pos in "ACU" and pos in "S":
      adj_noun_list += [(prev_word, word)]
    prev_pos = pos
    prev_word = word

against_pairs = FreqDist(adj_noun_list)

In [ ]:
for pair, count in against_pairs.most_common():
  print(f"{count: <3} {pair}")

8   ('illegaalsete', 'immigrantide')
8   ('odava', 'tööjõu')
7   ('konservatiivne', 'rahvaerakond')
4   ('massilise', 'sisserände')
4   ('uute', 'uudiste')
4   ('illegaalse', 'immigratsiooni')
3   ('uus', 'valitsus')
3   ('uued', 'uudised')
3   ('illegaalseid', 'immigrante')
3   ('suur', 'probleem')
3   ('pikka', 'aega')
3   ('kristliku', 'taustaga')
3   ('ebaseaduslikke', 'migrante')
3   ('euroopalike', 'väärtuste')
3   ('kogu', 'aeg')
3   ('paremat', 'elu')
3   ('täie', 'rauaga')
3   ('pikemat', 'aega')
2   ('kogu', 'maailma')
2   ('suurt', 'hulka')
2   ('massiline', 'immigratsioon')
2   ('liberaalses', 'ühiskonnas')
2   ('tohutul', 'hulgal')
2   ('saabunud', 'põgenike')
2   ('poliitilise', 'islami')
2   ('ebaseadusliku', 'migratsiooni')
2   ('negatiivne', 'mõju')
2   ('tänuväärset', 'tööd')
2   ('viimasel', 'ajal')
2   ('poliitiline', 'nõunik')
2   ('liberaalse', 'demokraatia')
2   ('tugeva', 'surve')
2   ('suures', 'osas')
2   ('demograafilisi', 'probleeme')
2   ('hea', 'elu')
2   

In [ ]:
adj_noun_list = []
for i, row in data.iterrows():

  if row["stanceConsolidated"] <= 3:
    continue

  prev_pos, prev_word = "-", "-"
  for pos, word in zip(row["morph_analysis"].partofspeech, row["morph_analysis"].words.text):
    pos = pos[0]
    word = word.lower()
    if prev_pos in "ACU" and pos in "S":
      adj_noun_list += [(prev_word, word)]
    prev_pos = pos
    prev_word = word

supportive_pairs = FreqDist(adj_noun_list)

In [ ]:
for pair, count in supportive_pairs.most_common():
  print(f"{count: <3} {pair}")

3   ('ebaseadusliku', 'rände')
3   ('rahvusvahelist', 'kaitset')
2   ('rahvusvahelise', 'rändekava')
2   ('soolise', 'võrdõiguslikkuse')
2   ('avatud', 'algus')
2   ('salliva', 'õpikeskkonna')
2   ('kogu', 'maailmas')
2   ('suure', 'panuse')
2   ('globaalses', 'ränderaamistikus')
2   ('rassilise', 'diskrimineerimise')
2   ('kohalikud', 'inimesed')
2   ('ühise', 'tegevuskava')
2   ('rahvusvahelise', 'kaitse')
2   ('viibivad', 'välismaalased')
2   ('rahvusvaheline', 'rändeorganisatsioon')
2   ('lühiajalise', 'töötamise')
2   ('käesoleva', 'aasta')
2   ('sinine', 'äratus')
2   ('globaalne', 'ränderaamistik')
1   ('seotud', 'lood')
1   ('suurimaks', 'ohuks')
1   ('avatud', 'ühiskonnale')
1   ('avatud', 'näiteks')
1   ('migratsioonivastaseid', 'meeleolusid')
1   ('rassistlike', 'intsidentide')
1   ('viimastel', 'aastatel')
1   ('suure', 'hulga')
1   ('ksenofoobne', 'propaganda')
1   ('suurendavaid', 'välismaalaste')
1   ('siinsesse', 'arengusse')
1   ('seotud', 'kulud')
1   ('diplomeeritud'

In [ ]:
neg_lemma = []
pos_lemma = []
neu_lemma = []

for (first, second), count in against_pairs.most_common():
  if "immigra" in second or "rän" in second:
    print(first, end=", ")
    text = Text(first).tag_layer()
    lemma = text.morph_analysis.lemma[0][0]
    neg_lemma += [lemma]

print()
print()

for (first, second), count in supportive_pairs.most_common():
  if "immigra" in second or 'rän' in second:
    print(first, end=", ")
    text = Text(first).tag_layer()
    lemma = text.morph_analysis.lemma[0][0]
    print(lemma, end=", ")
    pos_lemma += [lemma]

print()
print()

for (first, second), count in neutral_pairs.most_common():
  if "immigra" in second or 'rän' in second:
    print(first, end=", ")
    text = Text(first).tag_layer()
    lemma = text.morph_analysis.lemma[0][0]
    print(lemma, end=", ")
    neu_lemma += [lemma]

illegaalsete, massilise, illegaalse, illegaalseid, massiline, massiline, range, võõraste, paarituhandelise, kontrollimatu, isikliku, illegaalsed, kasvav, seotud, globaalse, ebaseadusliku, negatiivne, kuritahtliku, globaalses, järgmine, tulevate, ähvardav, illegaalne, konservatiivset, rahvusvahelise, suur, lähtuvast, tormavaid, jätkuv, kontrollimatu, kogu, salakavalat, ohtliku, riiklik, illegaalsele, efektiivsest, tülikad, massilist, rekordkõrge, kriminaalsed, illegaalsed, karmiks, illegaalset, allaheitliku, võimaliku, illegaalse, uue, avantüristlik, senisele, illegaalsest, uusi, illegaalset, uus, tänast, sealsed, arvukad, võõraste, sarnane, üleeuroopalise, piiramatu, eiravad, tuntud, kõrge, tugevnev, uusi, lõtva, rahvusvahelise, massilise, kontrollimatu, peamisest, legaalsed, käivat, kahjulikku, potentsiaalseks, illegaalse, jahtivad, ohtlikke, ebaseaduslike, elavad, lõdva, ühine, islamiusulised, üksikuid, valimatule, uue, uus, toimuv, illegaalsele, illegaalse, tark, suunduvate, ebasead

In [ ]:
# remove duplicates and ambiguous adjectives
neg_lemma = set(neg_lemma) - set(neu_lemma)
pos_lemma = set(pos_lemma) - set(neu_lemma)

In [ ]:
intersection = neg_lemma.intersection(pos_lemma)
print(intersection)

{'lähtuv', 'kasvav', 'piiramatu'}


In [ ]:
# remove adjectives that were both in negative and positive
neg_lemma -= intersection
pos_lemma -= intersection

In [ ]:
negs = sorted(list(set(neg_lemma)))
for neg in negs:
  print(neg, end=", ")

agressiivne, allaheitlik, avantüristlik, efektiivne, elama, isiklik, islamiusuline, jahtiv, järgmine, jätkuv, kahjulik, kogu, konservatiivne, kriminaalne, kuritahtlik, käiv, kõrge, lõtv, ohtlik, paarituhandeline, potentsiaalne, range, rekordkõrge, riiklik, salakaval, sarnane, sealne, senine, seotud, suunduv, suvaline, tark, teisene, toimuv, tugevnev, tuntud, tülikas, valimatu, ähvardav, ühine, üksik, üleeuroopaline, 

In [ ]:
poss = sorted(list(set(pos_lemma)))
for pos in poss:
  print(pos, end=", ")

esitatud, hiiglaslik, inimlik, laiahaardeline, lubatud, noor, oluline, seaduslik, tõstatatud, vaba, väärikas, üleilmne, 

In [ ]:
def framing_bias(morph_layer, wordlist=[]):
  lemmas = list(morph_layer.lemma)
  lemma_list = [lemma[0].lower() for lemma in lemmas if lemma[0] not in stopword_list and len(lemma[0]) >= 2]
  lemma_string = " ".join(lemma_list)

  if "immigra" in lemma_string or "rän" in lemma_string:
    for word in wordlist:
      if word in lemma_string:
        return 1

  return 0

In [ ]:
data["framing_against"] = data["morph_analysis"].apply(framing_bias, wordlist=negs)
data["framing_support"] = data["morph_analysis"].apply(framing_bias, wordlist=poss)

In [ ]:
data[data["framing_against"] == 1]["stance"].value_counts()

stance
against       144
neutral        92
supportive     31
Name: count, dtype: int64

In [ ]:
data[data["framing_support"] == 1]["stance"].value_counts()

stance
neutral       60
against       43
supportive    33
Name: count, dtype: int64

In [ ]:
# download file (google drive)
from google.colab import drive
from google.colab import files

drive.mount("/content/drive")
data.to_csv("/content/drive/MyDrive/EstonianStanceDetection/notebooks/data.csv", index=False)
# files.download("data.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# local
data.to_csv("data.csv", index=False)